## Setup

In [5]:
# Install the packages
%pip install -r '/content/requirements.txt'
%pip install https://github.com/TimDettmers/bitsandbytes/releases/download/0.46.0/bitsandbytes-0.46.0-py3-none-manylinux2014_x86_64.whl
# # Make a directory for pretrained models
# !mkdir -p pretrained
# %cd pretrained

# # Download the model from Hugging Face into the directory
# !huggingface-cli download --resume-download \
#   --local-dir-use-symlinks False \
#   --local-dir InternVL3-78B \
#   OpenGVLab/InternVL3-78B

# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# toy_ds = '/content/drive/My Drive/datasets/toy_ds'

  Using cached flash_attn-2.8.0.post2.tar.gz (7.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.37.2-py3-none-any.whl.metadata (129 kB)
  Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached pillow-11.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached numpy-2.1.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached texttable-1.7.0-py2.py3-none-any.whl.metadata (9.8 kB)
  Using cached psutil-7.0.0-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (22 kB)
  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached ipython-8.37.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached jupyter_client-8.6.3-py3-none-any.whl.metada

ERROR: Invalid requirement: 'bitsandbytes==cuda121': Expected end or semicolon (after name and no valid version specifier)
    bitsandbytes==cuda121
                ^


KeyboardInterrupt: 

## Video

In [2]:
import os
import math
import torch
import numpy as np
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer, AutoConfig

# ─── Prompt ─────────────────────────────────────────────────
prompt_string = """
Please write one detailed but concise paragraph describing the following, based on the video frames or images above:
- The overall scene and weather conditions (e.g., rain, puddles).
- Any visible weapons or use of force:
  • Guns (how many and where)
  • Non-lethal force (e.g., hitting with fists, stones, or sticks)
- Vehicles in the scene:
  • Presence of military vehicles (specify type)
  • License plates or markings (quote exact text)
- Human activity:
  • Actions of uniformed personnel (include uniform colors)
  • Actions of protestors
  • Whether anyone appears to be injured, hurt, or lying on the ground
"""
# ─── Model Setup 78B 8 bit quant ─────────────────────────────────────────────────
MODEL_PATH = "/content/pretrained/InternVL3-78B"
model = AutoModel.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)

# ─── Constants ───────────────────────────────────────────────────
INPUT_SIZE = 448
NUM_FRAMES = 32
GEN_CONFIG = dict(max_new_tokens=1024, do_sample=True)
MEAN = (0.485, 0.456, 0.406)
STD = (0.229, 0.224, 0.225)

# ─── Split Model Across GPUs ──────────────────────────────────────────────
def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers

    # Distribute layers evenly, but give half of GPU0 to the vision part
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)

    layer_cnt = 0
    for gpu_idx, count in enumerate(num_layers_per_gpu):
        for _ in range(count):
            device_map[f'language_model.model.layers.{layer_cnt}'] = gpu_idx
            layer_cnt += 1

    # Pin all vision & shared embeddings to GPU0
    vision_keys = [
        'vision_model',
        'mlp1',
        'language_model.model.tok_embeddings',
        'language_model.model.embed_tokens',
        'language_model.output',
        'language_model.model.norm',
        'language_model.model.rotary_emb',
        'language_model.lm_head',
        f'language_model.model.layers.{num_layers - 1}'
    ]
    for key in vision_keys:
        device_map[key] = 0

    return device_map

# ─── Transforms ─────────────────────────────────────────────────
def build_transform():
    return T.Compose([
        T.Lambda(lambda img: img.convert('RGB')),
        T.Resize((INPUT_SIZE, INPUT_SIZE), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])

# ─── Load and Preprocess Video ──────────────────────────────────
def get_frame_indices(num_frames, total):
    return np.linspace(0, total - 1, num_frames, dtype=int)

def load_video(video_path, num_frames):
    vr = VideoReader(video_path, ctx=cpu(0))
    transform = build_transform()
    indices = get_frame_indices(num_frames, len(vr))
    pixel_values = [transform(Image.fromarray(vr[i].asnumpy())) for i in indices]
    return torch.stack(pixel_values)  # [num_frames, 3, H, W]

# ─── Inference ──────────────────────────────────────────────────
def infer(video_path):
    video_tensor = load_video(video_path, NUM_FRAMES).to(torch.bfloat16).cuda()
    prompt = ''.join([f'Frame{i+1}: <image>\n' for i in range(NUM_FRAMES)])
    prompt += prompt_string

    response, _ = model.chat(
        tokenizer,
        video_tensor,
        prompt,
        GEN_CONFIG,
        history=None,
        return_history=True
    )
    print("User:", prompt)
    print("Assistant:", response)

# ─── Entry ──────────────────────────────────────────────────────
if __name__ == "__main__":
    infer("/content/drive/MyDrive/toy_ds/videos/TNS_0169_V.mp4")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


FlashAttention2 is not installed.


None of the available devices `available_devices = None` are supported by the bitsandbytes version you have installed: `bnb_supported_devices = {'"cpu" (needs an Intel CPU and intel_extension_for_pytorch installed and compatible with the PyTorch version)', 'cuda', 'npu', 'mps', 'xpu', 'hpu'}`. Please check the docs to see if the backend you intend to use is available and how to install it: https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: None of the available devices `available_devices = None` are supported by the bitsandbytes version you have installed: `bnb_supported_devices = {'"cpu" (needs an Intel CPU and intel_extension_for_pytorch installed and compatible with the PyTorch version)', 'cuda', 'npu', 'mps', 'xpu', 'hpu'}`. Please check the docs to see if the backend you intend to use is available and how to install it: https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

## Image

In [ ]:
import math
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer, AutoConfig

# ─── Prompt ─────────────────────────────────────────────────
prompt_string = "Provide a detailed description of each image. Describe the foreground and background separately. Mention any people, objects, and actions clearly. What are the people doing? What expressions or activities are visible? What is the setting or context? Is there violence happening?"

# ─── Config ───────────────────────────────────────────────────────────────
MODEL_PATH = "./pretrained/InternVL3-78B"
INPUT_SIZE   = 448
MAX_PATCHES  = 12
GEN_CONFIG   = dict(max_new_tokens=1024, do_sample=True)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

# ─── Split Model Across GPUs ──────────────────────────────────────────────
def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers

    # Distribute layers evenly, but give half of GPU0 to the vision part
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)

    layer_cnt = 0
    for gpu_idx, count in enumerate(num_layers_per_gpu):
        for _ in range(count):
            device_map[f'language_model.model.layers.{layer_cnt}'] = gpu_idx
            layer_cnt += 1

    # Pin all vision & shared embeddings to GPU0
    vision_keys = [
        'vision_model',
        'mlp1',
        'language_model.model.tok_embeddings',
        'language_model.model.embed_tokens',
        'language_model.output',
        'language_model.model.norm',
        'language_model.model.rotary_emb',
        'language_model.lm_head',
        f'language_model.model.layers.{num_layers - 1}'
    ]
    for key in vision_keys:
        device_map[key] = 0

    return device_map

# ─── Image Preprocessing ─────────────────────────────────────────────────
def build_transform():
    return T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((INPUT_SIZE, INPUT_SIZE), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
    ])

def dynamic_preprocess(image, image_size=448, max_num=12):
    orig_w, orig_h = image.size
    aspect = orig_w / orig_h

    # find best grid (i × j) closest to aspect, with i*j ≤ max_num
    best, best_diff = (1,1), float('inf')
    for i in range(1, max_num+1):
        for j in range(1, max_num+1):
            if i*j > max_num: continue
            diff = abs(aspect - (i/j))
            if diff < best_diff:
                best, best_diff = (i,j), diff

    gw, gh = best
    new_w, new_h = image_size * gw, image_size * gh
    image = image.resize((new_w, new_h))

    tiles = []
    for y in range(gh):
        for x in range(gw):
            box = (x*image_size, y*image_size, (x+1)*image_size, (y+1)*image_size)
            tiles.append(image.crop(box))
    return tiles

def load_image(path, image_size=448, max_num=12):
    img = Image.open(path).convert('RGB')
    tiles = dynamic_preprocess(img, image_size=image_size, max_num=max_num)
    tfm = build_transform()
    return torch.stack([tfm(t) for t in tiles])

# ─── Inference ────────────────────────────────────────────────────────────
def infer(image_path):
    device_map = split_model(MODEL_PATH)
    model     = AutoModel.from_pretrained(
                    MODEL_PATH,
                    torch_dtype=torch.bfloat16,
                    low_cpu_mem_usage=True,
                    use_flash_attn=True,
                    trust_remote_code=True,
                    device_map=device_map
                ).eval()
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True, use_fast=False)

    pixel_values = load_image(image_path, max_num=MAX_PATCHES)
    pixel_values = pixel_values.to(torch.bfloat16).cuda()

    question = f"<image>\n{prompt_string}"
    response, _ = model.chat(tokenizer, pixel_values, question, GEN_CONFIG, history=None, return_history=True)

    print("User:", question)
    print("Assistant:", response)

if __name__ == "__main__":
    infer("toy_ds/images/TNS_3773_I.jpg")

/home/jupyter-dai7591/.conda/envs/videoqa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: <image>
Provide a detailed description of each image. Describe the foreground and background separately. Mention any people, objects, and actions clearly. What are the people doing? What expressions or activities are visible? What is the setting or context? Is there violence happening?
Assistant: **Foreground:**

1. **Person Standing:**
   - **Gender:** Male
   - **Appearance:** Short dark hair and wearing a light-colored, button-up shirt that is partially unbuttoned and draped loosely over his shoulders. He has dark pants with some damage and is barefoot.
   - **Limbs:** Both hands are visible but not directly touching; he is holding a book in his left hand.
   - **Attire:** The shirt is plain with some visible stains and patches, paired with darker jeans.
   - **Footwear:** Dark flip-flops.

2. **Actions:**
   - The individual is holding what seems to be a book or paper.

**Background:**

1. **People:**
   - There are several individuals visible in the background, appearing to 